In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 10, 15
from matplotlib.pyplot import imshow

import rawpy
import numpy as np
from spells import \
    smoothstep3, \
    smoothstart2, \
    smoothstart3, \
    smoothstart4, \
    smoothstart5, \
    smoothstart6, \
    smoothstop2, \
    smoothstop3, \
    smoothstop4, \
    smoothstop5, \
    smoothstop6, \
    crossfade

import PyOpenColorIO as OCIO

## Check that we have the correct OCIO config

In [ ]:
import os

print(os.environ["OCIO"])

## Some utility funtions

In [ ]:
ocio_config = OCIO.GetCurrentConfig()

ocio_display = ocio_config.getDefaultDisplay()
ocio_view = ocio_config.getDefaultView(ocio_display)
ocio_processor = ocio_config.getProcessor(
    OCIO.ROLE_SCENE_LINEAR,
    ocio_display,
    ocio_view,
    OCIO.TRANSFORM_DIR_FORWARD)
ocio_cpu = ocio_processor.getDefaultCPUProcessor()

def ccshow(image):
    """
    Applies a scene-linear to display colour space transform to a copy of the
    image before calling `imshow` on it
    """
    to_display = image.copy()
    ocio_cpu.applyRGB(to_display)

    print(f"max: {np.max(to_display)}")
    print(f"min: {np.min(to_display)}")

    imshow(to_display)

In [ ]:
srgb_processor = ocio_config.getProcessor(OCIO.ROLE_SCENE_LINEAR, "srgb8")
srgb_cpu = srgb_processor.getDefaultCPUProcessor()

def srgbshow(image):
    to_display = image.copy()
    srgb_cpu.applyRGB(to_display)

    to_display = to_display * 1.1344219641

    print(f"max: {np.max(to_display)}")
    print(f"min: {np.min(to_display)}")
    print(f"to_display.dtype: {to_display.dtype}")

    imshow(to_display)

In [ ]:
# test image for ccshow

img = np.zeros((256, 256, 3), np.float32)

# black grey white
img[:42, 85:170, ...] = 0.18
img[42:85, 85:170, ...] = 0.5
img[:85, 170:256, ...] = 1.0

# black-white gradient

for x in range(256):
    img[30:54, x, ...] = x/255

# red green blue
img[85:170, :85, 0] = 1
img[85:170, :85, 1] = 0
img[85:170, :85, 2] = 0

img[85:170, 85:170, 0] = 0
img[85:170, 85:170, 1] = 1
img[85:170, 85:170, 2] = 0

img[85:170, 170:256, 0] = 0
img[85:170, 170:256, 1] = 0
img[85:170, 170:256, 2] = 1.0

# cyan magenta yellow

img[170:256, :85, 0] = 0
img[170:256, :85, 1] = 1
img[170:256, :85, 2] = 1

img[170:256, 85:170, 0] = 1
img[170:256, 85:170, 1] = 0
img[170:256, 85:170, 2] = 1

img[170:256, 170:256, 0] = 1
img[170:256, 170:256, 1] = 1
img[170:256, 170:256, 2] = 0

imshow(img)


In [ ]:
ccshow(img)

In [ ]:
srgbshow(img)

# Load an image

Edit the string `img_path` to select your image.

In [ ]:
img_path = "../data/richard.iiq"

raw_data = rawpy.imread(img_path)

uncorrected_linear = raw_data.postprocess(
    half_size=True,
    output_color=rawpy.ColorSpace.raw,
    # output_bps=16,
    gamma=(1, 1),
    user_wb=[1.0, 1.0, 1.0, 1.0],
    no_auto_bright=True
    )

imshow(uncorrected_linear)

In [ ]:
x1 = 1000
x2 = 4300
y1 = 25
y2 = 200

wb_patch = uncorrected_linear[y1:y2, x1:x2]

imshow(wb_patch)

In [ ]:
avg_r = np.average(wb_patch[..., 0])
avg_g = np.average(wb_patch[..., 1])
avg_b = np.average(wb_patch[..., 2])

base_wb = [avg_g/avg_r, 1.0, avg_g/avg_b, 1.0]

white_balanced = np.array(raw_data.postprocess(
    user_wb=base_wb,
    # output_color=rawpy.ColorSpace.ProPhoto,
    output_color=rawpy.ColorSpace.raw,
    output_bps=16
    ), dtype=np.float32) / np.iinfo(np.uint16).max

ccshow(white_balanced)

In [ ]:
wb_proxy = white_balanced[::10, ::10]

ccshow(wb_proxy)

In [ ]:
working_data = wb_proxy # replace with `white_balanced` for full resolution

max_r = np.max(working_data[..., 0])
max_g = np.max(working_data[..., 1])
max_b = np.max(working_data[..., 2])

inverted = 1 - working_data.copy()

# inverted[..., 0] = max_r - working_data[..., 0]
# inverted[..., 1] = max_g - working_data[..., 1]
# inverted[..., 2] = max_b - working_data[..., 2]

ccshow(inverted)
imshow(inverted)


In [ ]:
corrected = inverted.copy()

# corrected[..., 0] = np.clip(corrected[..., 0] * 1.15, 0, 1.0)
# corrected[..., 1] = np.clip(corrected[..., 1] * 1.08, 0, 1.0)
# corrected[..., 2] = np.clip(corrected[..., 2] * 1.0, 0, 1.0)

# corrected = np.clip(corrected * 1.5, 0, 1.0)

ccshow(corrected)

In [ ]:
curved = corrected.copy()

offset = 0.42

# curved = np.clip((curved * (1 + offset)) - offset, 0, 1)

curved = crossfade(smoothstart5, smoothstop2, curved)

ccshow(curved)
imshow(curved)